In [2]:
import pandas as pd
from sklearn.metrics import roc_auc_score

import numpy as np



In [3]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import numpy as np

# Initialize a list to store AUC scores for all folds
all_auc_scores = []

for fold in range(0, 5):
    predictions = pd.read_csv(f"predictions_fold_{fold}.csv")
    y_test = pd.read_csv(f"y_test_fold_{fold}.csv")
    
    # Compute AUC for each task
    for column in y_test.columns:
        # Get the predictions and true values for the current task
        y_true = y_test[column].values
        y_pred = predictions[column].values

        # Filter out -1 values
        valid_indices = y_true != -1
        y_true_valid = y_true[valid_indices]
        y_pred_valid = y_pred[valid_indices]

        # Compute AUC if there are valid data points
        if len(y_true_valid) > 0 and len(np.unique(y_true_valid)) > 1:
            auc = roc_auc_score(y_true_valid, y_pred_valid)
        else:
            auc = np.nan  # Not enough data to compute AUC

        all_auc_scores.append({"Fold": fold, "Task": column, "AUC": auc})

# Convert to DataFrame for convenience
auc_df = pd.DataFrame(all_auc_scores)

# Save the DataFrame to a CSV file if needed
auc_df.to_csv("auc_scores_across_folds.csv", index=False)
auc_df

,Fold,Task,AUC
0,0,1_1,0.857143
1,0,10_19,NaN
2,0,10_20,NaN
3,0,100_277,0.375000
4,0,100_278,0.642857
...,...,...,...
1345,4,9_18,0.300000
1346,4,90_256,0.035714
1347,4,91_257,1.000000
1348,4,92_258,1.000000


In [12]:
mean_auc = auc_df.groupby("Task").mean().reset_index(drop=False)
mean_auc

,Task,Fold,AUC
0,100_277,2.0,0.651667
1,100_278,2.0,0.723016
2,104_289,2.0,0.669231
3,104_291,2.0,0.760897
4,104_295,2.0,0.714286
...,...,...,...
265,92_258,2.0,0.646429
266,93_259,2.0,0.506272
267,9_14,2.0,0.500000
268,9_17,2.0,1.000000


In [14]:
mean_auc[mean_auc["AUC"]==1]

,Task,Fold,AUC
43,115_409,2.0,1.0
63,133_474,2.0,1.0
65,134_477,2.0,1.0
96,181_599,2.0,1.0
97,181_600,2.0,1.0
131,200_658,2.0,1.0
221,29_43,2.0,1.0
244,78_237,2.0,1.0
268,9_17,2.0,1.0


In [5]:
len(mean_auc[mean_auc["AUC"]==1])

9

In [15]:
len(mean_auc[mean_auc["AUC"]>0.90])

18

In [16]:
mean_auc.to_csv("scaffold_mean_AUCSeal.csv")

In [17]:
median_auc = auc_df.groupby("Task").median().reset_index(drop=False)
median_auc.to_csv("scaffold_median_AUCSeal.csv")

In [18]:
len(median_auc[median_auc["AUC"]==1])

15

In [19]:
len(median_auc[median_auc["AUC"]>0.90])

24